In [ ]:
import cadquery as cq
from jupyter_cadquery.cadquery import show, show_accuracy, show_constraints, Assembly, Part, Faces
from jupyter_cadquery import set_sidecar

set_sidecar("Assembly")

**Some helpers**

In [ ]:
# Avoid clean error
cq.occ_impl.shapes.Shape.clean = lambda x: x


def L(x,y,z):
    return cq.Location(cq.Vector(x, y, z))

def C(*c):
    return cq.Color(*c)

def center(assy, name):
    return cq.Vertex.makeVertex(*assy.objects[name].obj.val().Center().toTuple())

def query_all(assy, q):
    name, kind, arg = q.split("@")

    tmp = cq.Workplane()
    obj = assy.objects[name].obj

    if isinstance(obj, (cq.Workplane, cq.Shape)):
        tmp.add(obj)
        res = getattr(tmp, kind)(arg)

    return res.objects if isinstance(res.val(), cq.Shape) else None

# 1 Bearing

## 1.1 Parts

In [ ]:
def ring(inner_radius, outer_radius, width):
    ring = (cq.Workplane(origin=(0, 0, -width / 2))
        .circle(outer_radius).circle(inner_radius)
        .extrude(width)
    )
    return ring

tol = 0.05
ball_diam = 5

r1, r2, r3, r4 = 4, 6, 8, 10
r5 = (r3 + r2) / 2
inner_ring = ring(r1, r2, ball_diam)
outer_ring = ring(r3, r4, ball_diam)

torus = cq.CQ(cq.Solid.makeTorus(r5, ball_diam / 2 + tol))
ball = cq.Workplane().sphere(ball_diam / 2)

inner = inner_ring.cut(torus)
outer = outer_ring.cut(torus)

show(Assembly([Part(ball, "ball"), Part(inner, "inner"), Part(outer, "outer")], "bearing"))

## 1.2 Assembly

In [ ]:
def balls(i):
    return "ball_%d" % i

number_balls = 6

def create_bearing(cls, helpers=True):
    assy = cls(outer, loc=L(0, 0, ball_diam/2), name="outer", color=C("orange"))
    assy.add(inner, loc=L(20, 0, 0), name="inner", color=C("orange"))
    for i in range(number_balls):
        assy.add(ball, loc=L(6*i, 20, 0), name=balls(i), color=C("black"))

    if helpers:
        assy.add(cq.Workplane().circle(1).extrude(1), loc=L(0,-20,0), name="_center")
        assy.add(cq.Workplane().polygon(number_balls, 2*r5).extrude(ball_diam/2), loc=L(20,-20,0), name="_points")

    return assy

## 1.3 Numerical solver from cadquery.Assembly

### 1.3.1 Assembly

In [ ]:
bearing = create_bearing(cq.Assembly)
show(bearing)

### 1.3.2 Constraints

In [ ]:
points = query_all(bearing, "_points@vertices@>Z")

cs = [
    ("outer@faces@<Z", "_center@faces@>Z", "Plane"),
    ("inner@faces@<Z", "_center@faces@>Z", "Plane"),
    ("_points@faces@<Z", "_center@faces@>Z", "Plane"),
] + [
    ("_points", points[i], balls(i), center(bearing, balls(i)), "Point") 
    for i in range(number_balls)
]

for c in cs:
    bearing.constrain(*c)
    
show_constraints(bearing, cs)

### 1.2.3 Solver

In [ ]:
bearing.solve()
show(bearing)

In [ ]:
show_accuracy(bearing, cs)

## 1.4 Mate Assembly from jupyter-cadquery
### 1.4.1 MAssembly

In [ ]:
from jupyter_cadquery.mate_assembly import Mate, MAssembly

bearing = create_bearing(MAssembly, helpers=False)
show(bearing)

### 1.4.2 Mates

In [ ]:
M = lambda *args: Mate(bearing.find(*args))

bearing.mate(
    name="outer", selector="outer", mate=M("outer", ("faces", "<Z")),
    is_origin=True,
).mate(
    name="inner", selector="inner", mate=M("inner", ("faces", "<Z")),
    is_origin=True,
)
for i in range(number_balls):
    bearing.mate(
        name=balls(i), selector=balls(i), mate=Mate((0,0,0), (1,0,0), (0,0,1)),
        is_origin=True
    ).mate(
        name="inner_%d" %i, selector="inner", mate=Mate((0,0,0), (1,0,0), (0,0,1))
                                                   .rz(i*60)
                                                   .tx(r5)
    )
show(bearing, render_mates=True)

### 1.4.2 Relocate and assemble

In [ ]:
bearing.relocate()

# assemble each part
bearing.assemble("inner", "outer")
for i in range(number_balls):
    bearing.assemble(balls(i), "inner_%d"%i)
show(bearing, render_mates=True, transparent=True)

# 2 Simple Tests

In [ ]:
def create():
    b1 = cq.Workplane().box(1, 1, 2).faces('>Z').edges('>X').chamfer(0.4)
    b2 = cq.Workplane().box(.1, 2, 1).faces('>Z').edges('>Y').chamfer(0.1)
    b3 = cq.Workplane().box(2, .1, .5).faces('>Z').edges('>X').chamfer(0.1)
    b4 = cq.Workplane().box(1, 1, .2).faces('>Y').edges('>X').chamfer(0.1)

    assy = cq.Assembly(b1, loc=L(1, 1, 0), name="TOP")
    assy2 = cq.Assembly(b2, name="SECOND")
    assy3 = cq.Assembly(b3, name="THIRD", color=C('orange'))

    assy.add(assy2, color=C("green"))
    assy.add(assy3)
    assy.add(b4, name="4th",color=C("blue1"))
    return assy

In [ ]:
assy = create()
cs = [
    ("TOP@faces@>(1,0,1)", "THIRD@faces@>Z", "Plane"), 
    ("TOP@faces@<Y", "SECOND@faces@<Y", "Axis"),
    ("THIRD@faces@<X", "SECOND@faces@>X", "Plane"),
    ("SECOND@faces@>Z", "THIRD@faces@<Z", "Axis"),
    ("4th@faces@>Z", "SECOND@faces@<Z", "Plane"),
    ("4th@faces@>Y", "TOP@faces@<Y", "Axis"),    
]
show_constraints(assy, cs)

In [ ]:
for c in cs:
    assy.constrain(*c)

assy.solve()
show(assy)

In [ ]:
show_constraints(assy, cs)

In [ ]:
show_accuracy(assy, cs)